In [ ]:
#%%
import numpy as np
import pandas as pd
import itertools as it
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from source.engine import funciones as f
from source.engine.InputsNoRevolvente import InputsNoRevolvente
from source.engine.OutputsNoRevolvente import OutputsNoRevolvente

ruta = r'C:\Users\joelo\Documents\Python\Monitoreo\Data'

nombreproducto = '\hipot'
REAL = pd.read_csv(ruta+str(nombreproducto)+'_real.csv')
TEORICO = pd.read_csv(ruta+str(nombreproducto)+'_inputs.csv')
PRECIOS = pd.read_csv(ruta+str(nombreproducto)+'_precios.csv')

inicio = 202101
fin = 202212
product = InputsNoRevolvente(REAL,TEORICO,mincosecha=inicio,maxcosecha=fin)


In [ ]:
lista_cortes = [['C_TOTAL'],
                ['C_QUARTER'],
                ['C_QUARTER','C_PLAZO'],['C_QUARTER','C_PLAZO','C_SEGMENTO'],
                ['C_QUARTER','C_SEGMENTO'],['C_QUARTER','C_SEGMENTO','C_PLAZO'],
                ['C_PLAZO'],
                ['C_PLAZO','C_QUARTER'],['C_PLAZO','C_QUARTER','C_SEGMENTO'],
                ['C_PLAZO','C_SEGMENTO'],['C_PLAZO','C_SEGMENTO','C_QUARTER'],
                ['C_SEGMENTO'],
                ['C_SEGMENTO','C_QUARTER'],['C_SEGMENTO','C_QUARTER','C_PLAZO'],
                ['C_SEGMENTO','C_PLAZO'], ['C_SEGMENTO','C_PLAZO','C_QUARTER']
                ]           


In [ ]:
lista_cortes = [['C_TOTAL']]
#product.curvasR['vivos']

In [ ]:
appended_curvas = []
for agregado_cortes in lista_cortes:

    product.condensar(agregado_cortes)
    product.optimizar()
    product.impacto(PRECIOS)

    curvas = pd.merge(left=product.curvas, right=product.Impacto[agregado_cortes 
                                                                 + ['ROA_base'] + ['ROA_delta_pd'] + ['ROA_delta_can'] + ['ROA_delta_pre'] + ['ROA_final']
                                                                 + ['TIR_base'] + ['TIR_delta_pd'] + ['TIR_delta_can'] + ['TIR_delta_pre'] + ['TIR_final']], 
                      how='left', left_on=agregado_cortes, right_on=agregado_cortes)
    
    if len(agregado_cortes) == 1:
        corte = curvas[agregado_cortes[0]].map(str)
    elif len(agregado_cortes) == 2:
        corte = curvas[agregado_cortes[0]].map(str) + '-' + curvas[agregado_cortes[1]].map(str)
    elif len(agregado_cortes) == 3:
        corte = curvas[agregado_cortes[0]].map(str) + '-' + curvas[agregado_cortes[1]].map(str) + '-' + curvas[agregado_cortes[2]].map(str)

    curvas['CORTE'] = corte
    curvas = curvas[ ['CORTE'] + [ col for col in curvas.columns if col != 'CORTE' ] ]

    appended_curvas.append(curvas)
    
appended_curvas = pd.concat(appended_curvas, ignore_index=True)

In [ ]:
dicts = {}
keys = appended_curvas.CORTE
keys2 = ['pd_real', 'can_real', 'pre_real', 'pd_teorico', 'can_teorico','pre_teorico']
values = {'pd_real': ['PD','REAL'], 'can_real': ['Cancelaciones','REAL'], 'pre_real': ['Prepagos','REAL'],
          'pd_teorico': ['PD','TEORICO'], 'can_teorico': ['Cancelaciones','TEORICO'], 'pre_teorico': ['Prepagos','TEORICO']}

for i in keys.index:
    dicts[keys[i]] = values


In [ ]:
"""  columns = ['CORTE','MODELO','CURVA','MADURACION','VALOR']
df = pd.DataFrame(columns=columns)
d = {}

for key in keys:
    for key2 in keys2:
        d[key,key2] = pd.DataFrame(columns=columns)
        d[key,key2]['VALOR'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,key2]
        d[key,key2]['MODELO'] = dicts[key][key2][0]
        d[key,key2]['CURVA'] = dicts[key][key2][1]
        d[key,key2]['MADURACION'] = d[key,key2].index + 1
        d[key,key2]['CORTE'] = key"""

In [ ]:
columns = ['CORTE','OPERACIONES','OPE_MADURACION','MADURACION',
           'PD_REAL','PD_TEORICO','CAN_REAL','CAN_TEORICO','PREPAGO_REAL','PREPAGO_TEORICO',
           'ROA_base','ROA_delta_pd','ROA_delta_can','ROA_delta_pre','ROA_final',
           'TIR_base','TIR_delta_pd','TIR_delta_can','TIR_delta_pre','TIR_final']
df = pd.DataFrame(columns=columns)
d = {}

for key in keys:

    d[key] = pd.DataFrame(columns=columns)
    d[key]['PD_REAL'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'pd_real'] 
    d[key]['PD_TEORICO'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'pd_teorico'] 
    d[key]['CAN_REAL'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'can_real'] 
    d[key]['CAN_TEORICO'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'can_teorico'] 
    d[key]['PREPAGO_REAL'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'pre_real'] 
    d[key]['PREPAGO_TEORICO'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'pre_teorico'] 
    d[key]['OPE_MADURACION'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'vivos'] 
    d[key]['MADURACION'] = d[key].index + 1
    d[key]['ROA_base'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'ROA_base']
    d[key]['ROA_delta_pd'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'ROA_delta_pd']
    d[key]['ROA_delta_can'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'ROA_delta_can']
    d[key]['ROA_delta_pre'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'ROA_delta_pre']
    d[key]['ROA_final'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'ROA_final']
    d[key]['TIR_base'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'TIR_base']
    d[key]['TIR_delta_pd'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'TIR_delta_pd']
    d[key]['TIR_delta_can'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'TIR_delta_can']
    d[key]['TIR_delta_pre'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'TIR_delta_pre']
    d[key]['TIR_final'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'TIR_final']
    d[key]['OPERACIONES'] = appended_curvas.loc[appended_curvas['CORTE'] == key].reset_index(drop=True).at[0,'recuento']
    d[key]['CORTE'] = key

In [ ]:
pd.concat({k: pd.DataFrame(v) for k, v in d.items()}, axis=0, ignore_index=True).to_excel('monitoreo.xlsx')